# CNN development
## Data retrieval

Mounting the data drive.

In [1]:
!apt-get install unrar
!pip install split_folders
!pip install --upgrade pip
!pip install --upgrade Pillow
!pip install tensorflow_datasets
!pip install -U scikit-learn

'''
!mkdir "content"
!mkdir "content/sample_data"
!unrar x "augmented/dataset.rar" -d "content/sample_data"
!mkdir "content/simple_data"
!unrar x "simple/dataset.rar" -d "content/simple_data"
'''

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.5.8-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


'\n!mkdir "content"\n!mkdir "content/sample_data"\n!unrar x "augmented/dataset.rar" -d "content/sample_data"\n!mkdir "content/simple_data"\n!unrar x "simple/dataset.rar" -d "content/simple_data"\n'

Splitting dataset into training, validation, and test subsets.

In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import splitfolders as sf
from tensorflow.compat.v1 import ConfigProto, InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

img_size = 299

val_split = 0.15

data_dir = pathlib.Path("content/sample_data/split/train")
batch_size = 1
img_height = img_size
img_width = img_size

#sf.ratio("content/sample_data", output="content/sample_data/split", seed=123, ratio=(.8, .2))

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=val_split,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=val_split,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  pathlib.Path("content/sample_data/split/val"),
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1744 files belonging to 3 classes.
Using 1483 files for training.
Found 1744 files belonging to 3 classes.
Using 261 files for validation.
Found 437 files belonging to 3 classes.


Acquiring training labels and images from dataset.

In [3]:
import numpy as np

train_y = []
train_X = []

for images, labels in train_ds:
    train_X.append(images.numpy()[0])
    train_y.append(labels.numpy()[0])
    
train_y = np.array(train_y)
train_X = np.array(train_X)

## Auxiliar functions

Parsing hyperparametres to String.

In [4]:
def hyperToString(hyper):
  cad = "model"
  i = 0
  for item in hyper:
    tmp = "_P"+str(i)+"-"+str(item)
    cad += tmp

  return cad

Cross-validation by K-folds.

In [5]:
from sklearn.model_selection import KFold, StratifiedKFold

def KFcrossVal(cnn,hyper,y_train,X_train):
  nSplits = 10
  randomState = 0
  shuffle = True

  print("-------",hyperToString(hyper))

  skf = StratifiedKFold(n_splits = nSplits, random_state = randomState, shuffle = shuffle)

  VALIDATION_ACCURACY = []
  VALIDATION_LOSS = []

  for train_index, val_index in skf.split(np.zeros(len(y_train)),y_train):
    trainingData_y = y_train.take(train_index,axis=0)
    trainingData_X = X_train.take(train_index,axis=0)
    validationData_y = y_train.take(val_index,axis=0)
    validationData_X = X_train.take(val_index,axis=0)

    training_data = tf.data.Dataset.from_tensor_slices((trainingData_X, trainingData_y))
    training_data = training_data.batch(32)

    validation_data = tf.data.Dataset.from_tensor_slices((validationData_X, validationData_y))
    validation_data = validation_data.batch(32)

    results = cnn(hyper[0],hyper[1],hyper[2],training_data,validation_data)

    VALIDATION_ACCURACY.append(results['binary_accuracy'])
    VALIDATION_LOSS.append(results['loss'])

  accuracyMean = np.sum(VALIDATION_ACCURACY) / len(VALIDATION_ACCURACY)
  lossMean = np.sum(VALIDATION_LOSS) / len(VALIDATION_LOSS)

  print("mean loss: ",lossMean )
  print("mean accuracy: ",accuracyMean)

  return [lossMean], [hyperToString(hyper)]

## Training
### Model Architecture

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import Xception, VGG16
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import datetime, os
import glob
from tensorflow.keras import regularizers
from sklearn.metrics import f1_score, balanced_accuracy_score, roc_auc_score, log_loss, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold

def cnnVGG16(l1,l2,epoch,train,val):
  base_model = VGG16(
    weights="imagenet",	# Load weights pre-trained on ImageNet.
    input_shape=(512, 512, 3),
    include_top=False,
  )
  base_model.trainable = False

  inputs = Input(shape=(img_height, img_width, 3))
  outputs = base_model(inputs,training=False)

  outputs = GlobalAveragePooling2D()(outputs)
  outputs = Dense(512,activation='relu',activity_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(outputs)
  outputs = Dense(256,activation='relu',activity_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(outputs)
  outputs = Dense(128,activation='relu',activity_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(outputs)
  outputs = Dense(1,activation='sigmoid',activity_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(outputs)

  model = keras.Model(inputs,outputs)

  model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[keras.metrics.BinaryAccuracy()],
  )

  model.fit(train, epochs=epoch, validation_data=val)

  results = model.evaluate(val)
  results = dict(zip(model.metrics_names,results))
  tf.keras.backend.clear_session()
  return results

def cnnXceptCnn(l1,l2,epoch,train,val):

  base_model = Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=False,
  )
  base_model.trainable = False
  inputs = Input(shape=(299, 299, 3))

  norm_layer = keras.layers.experimental.preprocessing.Normalization()
  mean = np.array([127.5] * 3)
  var = mean ** 2

  outputs = norm_layer(inputs)
  norm_layer.set_weights([mean, var])


  outputs = base_model(outputs,training=False)
  outputs = GlobalAveragePooling2D()(outputs)
  outputs = Dense(1024, activation='relu',activity_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(outputs)
  outputs = Dense(512, activation='relu',activity_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(outputs)
  outputs = Dense(2, activation='relu',activity_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(outputs)
  outputs = Dense(1, activation='sigmoid',activity_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(outputs)

  model = Model(inputs,outputs)

  model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[keras.metrics.BinaryAccuracy()],
  )

  epochs = int(epoch)
  model.fit(train, epochs=epoch, validation_data=val)

  results = model.evaluate(val)
  results = dict(zip(model.metrics_names,results))
  tf.keras.backend.clear_session()
  return results

In [7]:
def loops(cnn, hyper_parameters, variable_arguments, steps, index,values,labels):
  if(len(variable_arguments) == 0):
    return KFcrossVal(cnn,hyper_parameters,train_y,train_X)
  
  hyper_parameters = np.concatenate((hyper_parameters,[0]))
  
  for variable in np.arange(variable_arguments[0][0], variable_arguments[0][1] + steps[index], steps[index]):
    hyper_parameters[index] = variable
    result_value, result_label = loops(cnn,hyper_parameters,variable_arguments[1:],steps,index + 1,values,labels)
    print("results-----------------")
    print(result_value, result_label)
    values = np.concatenate((values, result_value))
    labels = np.concatenate((labels, result_label))
  
  return values, labels

# def XceptCnn(drop, learn, epoch, train, val):
values = []
labels = []

hyper_parameters_range = [[0.01,0.01],[0.01,0.01],[30,30]] # l1, l2, epochs
steps =                  [0.01,0.01,1]

valores, etiquetas = loops(cnnXceptCnn,[],hyper_parameters_range,steps,0,values,labels)
values, tags = loops(cnnVGG16,[],hyper_parameters_range,steps,0,values,labels)

'''hyper_parameters_range = [[0.48,0.49]]
steps =                  [0.01]
valores, etiquetas = loops(XceptCnn,[],hyper_parameters_range,steps,0,values,labels)'''

------- model_P0-0.01_P0-0.01_P0-30.0


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [ ]:
print(valores)
print(etiquetas)

print(values)
print(tags)
#print(XceptCnn(0.5,0.001,10,train_dataset_chido,val_dataset_chido))

#KFcrossVal(XceptCnn,[0.5,0.01,30],train_y,train_X)
